In [1]:
# 1. Instalar cliente de Kaggle e idx2numpy (para leer IDX fácil)
!pip -q install kaggle idx2numpy

# 2. Subir tu kaggle.json (al ejecutar abre un selector de archivos)
from google.colab import files
files.upload()  # <-- Seleccioná kaggle.json

# 3. Guardarlo con permisos correctos
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 4. Descargar el dataset de MNIST (autor: hojjatk)
!mkdir -p /content/mnist
!kaggle datasets download -d hojjatk/mnist-dataset -p /content/mnist

# 5. Descomprimir
!unzip -q /content/mnist/*.zip -d /content/mnist

# 6. Ver qué archivos quedaron
!ls -lh /content/mnist


  Preparing metadata (setup.py) ... done


Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/hojjatk/mnist-dataset
License(s): copyright-authors
  0% 0.00/22.0M [00:00<?, ?B/s]
100% 22.0M/22.0M [00:00<00:00, 962MB/s]
total 75M
-rw-r--r-- 1 root root  23M Oct 17  2019 mnist-dataset.zip
drwxr-xr-x 2 root root 4.0K Sep  3 22:31 t10k-images-idx3-ubyte
-rw-r--r-- 1 root root 7.5M Oct 17  2019 t10k-images.idx3-ubyte
drwxr-xr-x 2 root root 4.0K Sep  3 22:31 t10k-labels-idx1-ubyte
-rw-r--r-- 1 root root 9.8K Oct 17  2019 t10k-labels.idx1-ubyte
drwxr-xr-x 2 root root 4.0K Sep  3 22:31 train-images-idx3-ubyte
-rw-r--r-- 1 root root  45M Oct 17  2019 train-images.idx3-ubyte
drwxr-xr-x 2 root root 4.0K Sep  3 22:31 train-labels-idx1-ubyte
-rw-r--r-- 1 root root  59K Oct 17  2019 train-labels.idx1-ubyte


In [2]:
import idx2numpy
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim

# Rutas
base = "/content/mnist"
train_images = idx2numpy.convert_from_file(f"mnist/train-images-idx3-ubyte/train-images-idx3-ubyte")
train_labels = idx2numpy.convert_from_file(f"/content/mnist/train-labels-idx1-ubyte/train-labels-idx1-ubyte")
test_images  = idx2numpy.convert_from_file(f"/content/mnist/t10k-images-idx3-ubyte/t10k-images-idx3-ubyte")
test_labels  = idx2numpy.convert_from_file(f"/content/mnist/t10k-labels-idx1-ubyte/t10k-labels-idx1-ubyte")

# Normalizar y aplanar (28x28 → 784)
x_train = train_images.astype("float32") / 255.0
x_test  = test_images.astype("float32") / 255.0

x_train = x_train.reshape(-1, 28*28)
x_test  = x_test.reshape(-1, 28*28)

y_train = train_labels.astype("int64")
y_test  = test_labels.astype("int64")

# Convertir a tensores
x_train_t = torch.tensor(x_train)
y_train_t = torch.tensor(y_train)
x_test_t  = torch.tensor(x_test)
y_test_t  = torch.tensor(y_test)

# Crear datasets y dataloaders
train_dataset = TensorDataset(x_train_t, y_train_t)
test_dataset  = TensorDataset(x_test_t, y_test_t)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=128, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
model = nn.Sequential(
    nn.Linear(28*28, 512),  # capa entrada → 512
    nn.ReLU(),
    nn.Linear(512, 256),    # capa oculta → 256
    nn.ReLU(),
    nn.Linear(256, 10)      # salida → 10 clases
)

model = model.to(device)
print(model)


Sequential(
  (0): Linear(in_features=784, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=256, bias=True)
  (3): ReLU()
  (4): Linear(in_features=256, out_features=10, bias=True)
)


In [4]:
criterion = nn.CrossEntropyLoss()          # Para clasificación multiclase
optimizer = optim.Adam(model.parameters(), lr=1e-3)


In [5]:
def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss, correct, total = 0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.4f}, "
              f"Accuracy: {100*correct/total:.2f}%")

train_model(model, train_loader, criterion, optimizer, epochs=10)


Epoch [1/10], Loss: 0.2886, Accuracy: 91.81%
Epoch [2/10], Loss: 0.1040, Accuracy: 96.86%
Epoch [3/10], Loss: 0.0672, Accuracy: 97.94%
Epoch [4/10], Loss: 0.0477, Accuracy: 98.44%
Epoch [5/10], Loss: 0.0335, Accuracy: 98.94%
Epoch [6/10], Loss: 0.0269, Accuracy: 99.14%
Epoch [7/10], Loss: 0.0213, Accuracy: 99.32%
Epoch [8/10], Loss: 0.0178, Accuracy: 99.39%
Epoch [9/10], Loss: 0.0147, Accuracy: 99.52%
Epoch [10/10], Loss: 0.0131, Accuracy: 99.53%


In [6]:
def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {100*correct/total:.2f}%")

evaluate_model(model, test_loader)


Test Accuracy: 97.90%
